In [1]:
# 测试脚本
# 1.support set每一类的图片输入特征提取网络获取embedding
# 2.计算这一类的平均值，再归一化得到u1,u2,u3
# 3.query输入预训练网络得到特征向量，归一化，然后与u1,u2,u3堆成矩阵M乘积得到余弦相似度

### 计算两个向量的余弦相似度
$$cos(\theta)=\frac{ a \cdot b}{||a||\times||b||} $$

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import random
import glob

# 这里image需要重命名一下
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet import preprocess_input

In [3]:
embedding = tf.keras.models.load_model('./latest_new.h5')

In [4]:
# embedding.summary()

In [5]:
def extractFeat(model,filename):
    # 和训练时一样的数据预处理
    img = keras_image.load_img(filename, target_size= (200, 200) )
    img = keras_image.img_to_array(img)
    img = preprocess_input(img)
    # 扩张维度
    image = np.expand_dims(img, axis=0)
    # 预测
    feat = model.predict(image)
    # L2归一化
    norm_feat = feat[0] / np.linalg.norm(feat[0])
    return norm_feat

In [6]:
# 获取测试的support set 和query

In [7]:
query_classes = glob.glob('dogImages/train/*')[-5:]

In [8]:
query_classes

['dogImages/train\\129.Tibetan_mastiff',
 'dogImages/train\\130.Welsh_springer_spaniel',
 'dogImages/train\\131.Wirehaired_pointing_griffon',
 'dogImages/train\\132.Xoloitzcuintli',
 'dogImages/train\\133.Yorkshire_terrier']

In [9]:
query_files = []
# 遍历每一类
for class_item in query_classes:
    # 遍历下面所有文件
    query_file_list = glob.glob(class_item + '/*')
    query_files.append(query_file_list)


In [10]:
# query_files

In [11]:
# 遍历每一类，选择其中前两个作为support set计算平均值，第三个当成查询用query
class_u = []
for class_images in query_files:
    # 获取该类图片
    feat_A = extractFeat(embedding,class_images[0]) 
    feat_B = extractFeat(embedding,class_images[1])     
    # 平均值
    feat_avg = (feat_A + feat_B) / 2
    # 归一化
    feat_norm = feat_avg / np.linalg.norm(feat_avg)
    class_u.append(feat_norm)
# 转为numpy数组    
class_u = np.array(class_u)

In [12]:
class_u.shape

(5, 256)

In [13]:
class_u

array([[ 0.07781971, -0.08984091,  0.01712802, ...,  0.07326919,
         0.11925194, -0.00619248],
       [ 0.10244017, -0.06784289,  0.01928079, ..., -0.00323422,
         0.0917083 ,  0.05585233],
       [ 0.12418903, -0.12944539,  0.01605211, ..., -0.01283686,
         0.07236631,  0.08687896],
       [ 0.10840193, -0.12153535, -0.00541702, ..., -0.02355012,
        -0.00940335,  0.087505  ],
       [ 0.10187978, -0.10389711,  0.08479862, ..., -0.01444715,
         0.06643759,  0.02874403]], dtype=float32)

In [14]:
# 随机选一个query图片
query_img = query_files[4][5]

In [15]:
query_img

'dogImages/train\\133.Yorkshire_terrier\\Yorkshire_terrier_08319.jpg'

In [16]:
# 获取特征
query_feat = extractFeat(embedding,query_img)

In [17]:
# query_feat

In [18]:
# 计算余弦相似度
class_prob = np.dot(class_u,query_feat)

In [19]:
class_prob

array([0.3206181 , 0.5340429 , 0.9036767 , 0.71948004, 0.95542026],
      dtype=float32)

In [20]:
# 查询对应类别
query_classes[np.argmax(class_prob)]

'dogImages/train\\133.Yorkshire_terrier'

In [21]:
# 测试猫咪

In [22]:
labels = ['Abyssinian','Aegean','British_Shorthair','Donskoy','Persian']

In [23]:
# 遍历每一类
class_u = []
for label in labels:
    # 获取该类图片
    files = glob.glob('./cat/'+label+'/*')
    feats = [extractFeat(embedding,test_file) for test_file in files]
    # 平均值
    feat_avg = (feats[0] + feats[1]) /2 
    # 归一化
    feat_norm = feat_avg / np.linalg.norm(feat_avg)
    class_u.append(feat_norm)
    
class_u = np.array(class_u)

In [24]:
class_u.shape

(5, 256)

In [25]:
query_list = glob.glob('./cat/*_query*')

In [26]:
query_list

['./cat\\Abyssinian_query.jpg',
 './cat\\Aegean_query.jpg',
 './cat\\British_Shorthair_query.jpeg',
 './cat\\Donskoy_query.jpeg',
 './cat\\Persian_query.jpeg']

In [27]:
query_feat = extractFeat(embedding,query_list[3])

In [28]:
class_prob = np.dot(class_u,query_feat)

In [29]:
class_prob

array([0.74036133, 0.8545126 , 0.84924275, 0.974537  , 0.68840843],
      dtype=float32)

In [30]:
labels[np.argmax(class_prob)]

'Donskoy'